In [1]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, io, img_as_ubyte
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops_table
from skimage.segmentation import find_boundaries, mark_boundaries
import pandas as pd

from glob import glob

from stardist.models import StarDist2D 
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize

In [2]:
# Separate the individual stains from the IHC image
def color_separate(ihc_rgb):

    #Convert the RGB image to HED using the prebuilt skimage method
    ihc_hed = rgb2hed(ihc_rgb)
    
    # Create an RGB image for each of the separated stains
    #Convert them to ubyte for easy saving to drive as an image
    null = np.zeros_like(ihc_hed[:, :, 0])
    ihc_h = img_as_ubyte(hed2rgb(np.stack((ihc_hed[:, :, 0], null, null), axis=-1)))
    ihc_e = img_as_ubyte(hed2rgb(np.stack((null, ihc_hed[:, :, 1], null), axis=-1)))
    ihc_d = img_as_ubyte(hed2rgb(np.stack((null, null, ihc_hed[:, :, 2]), axis=-1)))

    #Optional fun exercise of combining H and DAB stains into a single image with fluorescence look
    
    h = rescale_intensity(ihc_hed[:, :, 0], out_range=(0, 1),
                          in_range=(0, np.percentile(ihc_hed[:, :, 0], 99)))
    d = rescale_intensity(ihc_hed[:, :, 2], out_range=(0, 1),
                          in_range=(0, np.percentile(ihc_hed[:, :, 2], 99)))

# Cast the two channels into an RGB image, as the blue and green channels
#Convert to ubyte for easy saving as image to local drive
    zdh = img_as_ubyte(np.dstack((null, d, h))) #DAB in green and H in Blue

    return (ihc_h, ihc_e, ihc_d, zdh)

In [3]:
# prints a list of available models 
StarDist2D.from_pretrained()
model = StarDist2D.from_pretrained('2D_versatile_he')
# model_2 = StarDist2D.from_pretrained('2D_versatile_fluo')

There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


ANALYZE ALL DATA

In [4]:
unlabel_img_dir = "../data/morphology_data/cropped_images/"
unlabel_images = glob(unlabel_img_dir+"*tif")

In [14]:
stats = pd.DataFrame(columns = ['img', 'label', 'area', 'perimeter'])
save_dir = '../data/morphology_data/cropped_auto_labeled/'
for img in unlabel_images:
    ihc_rgb=io.imread(img)
    if ihc_rgb.shape[-1] == 4:
        ihc_rgb = ihc_rgb[:, :, :3]
    H,E,D,HD = color_separate(ihc_rgb)
    H_labels, H_details = model.predict_instances(normalize(H))
    marked = mark_boundaries(ihc_rgb, H_labels, color=(1,0,0))
    fname = save_dir+img.split('\\')[1][:-4]+"_SEGMENTED.jpeg"
    io.imsave(fname, marked, quality = 100)
    H_props = regionprops_table(H_labels, H, 
                          properties=['label',
                                      'area', 'perimeter'])
    H_analysis_results = pd.DataFrame(H_props)
    H_analysis_results['img'] = img.split('\\')[1]
    stats = pd.concat([stats, H_analysis_results])
stats.to_csv(save_dir+'measurements_from_stardist.csv', index=False)

1/1 [==============================] - 0s 77ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


1/1 [==============================] - 0s 79ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


1/1 [==============================] - 0s 60ms/step


...

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


1/1 [==============================] - 0s 169ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


1/1 [==============================] - 0s 161ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
